## Part 1

In [1]:
from collections import deque

In [2]:
with open("input.txt") as f:
    data = f.read().split("\n\n")

In [3]:
def create_player_data(player: str) -> tuple:
    
    player = player.split("\n")
    
    player_num = player[0][-2]
    
    cards  = [int(e) for e in player[1:]]
    
    return player_num, cards

In [4]:
data

['Player 1:\n28\n13\n25\n16\n38\n3\n14\n6\n29\n2\n47\n20\n35\n43\n30\n39\n21\n42\n50\n48\n23\n11\n34\n24\n41',
 'Player 2:\n27\n37\n9\n10\n17\n31\n19\n33\n40\n12\n32\n1\n18\n36\n49\n46\n26\n4\n45\n8\n15\n5\n44\n22\n7']

In [7]:
class Player(deque):
    def draw_card(self):

        return self.popleft()


    def add_cards(self, cards):

        for card in cards:

            self.append(card)

        return self

In [8]:
Player([1, 2, 3]).add_cards([4, 5, 6])

Player([1, 2, 3, 4, 5, 6])

In [12]:
player_1 = Player(create_player_data(data[0])[1])

player_1.num = 1

player_2 = Player(create_player_data(data[1])[1])

player_2.num = 2

In [13]:
len(player_1) + len(player_2)

50

In [14]:
class Game:
    def __init__(self, player_1: Player, player_2: Player):

        self.player_1 = player_1
        self.player_2 = player_2

        self.winner = None

    def do_round(self):

        card_1 = self.player_1.draw_card()

        card_2 = self.player_2.draw_card()

        if card_1 > card_2:

            self.player_1.add_cards([card_1, card_2])

        elif card_2 > card_1:

            self.player_2.add_cards([card_2, card_1])
            
        else:
            
            raise ValueError(f"{card1, card_2}")

    def check_any_player_finished(self):

        if len(self.player_1) == 0:

            self.winner = self.player_2

            return True

        elif len(self.player_2) == 0:

            self.winner = self.player_1

            return True

        else:

            return False

    def play(self):

        while not self.check_any_player_finished():

            self.do_round()

        print(f"Winner's score: {self.calculate_score(self.winner)}")
        
        return self.winner.num

    def calculate_score(self, player):

        assert self.winner

        score = 0

        for idx, value in enumerate(list(reversed(player)), start=1):

            val = idx * value

            score += val

        return score

In [15]:
g = Game(player_1, player_2)

In [16]:
# enumerate(list(reversed(g.winner))=

In [17]:
g.play() # 33694

Winner's score: 33694


1

## Part 2

In [21]:
from collections import deque
from itertools import cycle

In [22]:
with open("input.txt") as f:
    data = f.read().split("\n\n")

In [23]:
def create_player_data(player: str) -> tuple:
    
    player = player.split("\n")
    
    player_num = player[0][-2]
    
    cards  = [int(e) for e in player[1:]]
    
    return player_num, cards

In [24]:
data

['Player 1:\n28\n13\n25\n16\n38\n3\n14\n6\n29\n2\n47\n20\n35\n43\n30\n39\n21\n42\n50\n48\n23\n11\n34\n24\n41',
 'Player 2:\n27\n37\n9\n10\n17\n31\n19\n33\n40\n12\n32\n1\n18\n36\n49\n46\n26\n4\n45\n8\n15\n5\n44\n22\n7']

In [25]:
class Player(deque):
    def draw_card(self):

        return self.popleft()

    def add_cards(self, cards):

        for card in cards:

            self.append(card)

        return self

In [26]:
player_1 = Player(create_player_data(data[0])[1])

player_1.num = 1

player_2 = Player(create_player_data(data[1])[1])

player_2.num = 2

In [27]:
len(player_1) + len(player_2)

50

In [30]:
class Game:
    def __init__(
        self,
        player_1: Player,
        player_2: Player,
        is_sub: bool = False,
        verbose: bool = False,
    ):

        self.player_1 = player_1
        self.player_2 = player_2

        self.winner = None

        self.is_sub = is_sub

        self.verbose = verbose

        self.configurations_seen = set()

    def do_round(self):

        # Probably faster than using lists
        configuration = tuple(
            zip(self.player_1, cycle(self.player_2))
            if len(self.player_1) > len(self.player_2)
            else zip(cycle(self.player_1), self.player_2)
        )

        # [list(self.player_1), list(self.player_2)]

        if configuration in self.configurations_seen:

            if self.verbose:
                print("Found previous configuration")
                # print(configuration, "\n")
                # print(self.configurations_seen)

            self.winner = self.player_1

            # make player 2 empty so that check_any_player_finished = True
            self.player_2 = Player([])

            return

        else:

            self.configurations_seen.add(configuration)

        card_1 = self.player_1.draw_card()

        card_2 = self.player_2.draw_card()

        if self.should_jump_into_subgame(card_1, card_2):

            new_player_1 = Player(list(self.player_1.copy())[:card_1])

            new_player_1.num = 1

            new_player_2 = Player(list(self.player_2.copy())[:card_2])

            new_player_2.num = 2

            if self.verbose:

                print(f"\n\nPlaying subgame {card_1}, {card_2}")

                print(new_player_1)

                print(new_player_2)

            new_game = self.__class__(new_player_1, new_player_2, is_sub=True)

            sub_winner = new_game.play()

            if self.verbose:

                print(f"Sub winner:", sub_winner)

            if sub_winner == 1:

                self.player_1.add_cards([card_1, card_2])

            elif sub_winner == 2:

                self.player_2.add_cards([card_2, card_1])

            return

        if card_1 > card_2:

            self.player_1.add_cards([card_1, card_2])

            return

        elif card_2 > card_1:

            self.player_2.add_cards([card_2, card_1])

            return

        else:

            raise ValueError(f"{card1, card_2}")

    def check_any_player_finished(self):

        if len(self.player_1) == 0:

            self.winner = self.player_2

            return True

        elif len(self.player_2) == 0:

            self.winner = self.player_1

            return True

        else:

            return False

    def play(self):

        while not self.check_any_player_finished():

            self.do_round()

        if self.is_sub is False:

            print(f"Winner's score: {self.calculate_score(self.winner)}")

        return self.winner.num

    def should_jump_into_subgame(self, card_player_1, card_player_2):

        if self.verbose:

            print(f"==\nCard player 1 {card_player_1}")

            print(f"Len player 1: {len(self.player_1)}")

            print(f"Card player 2 {card_player_2}")

            print(f"Len player 2: {len(self.player_2)}\n==\n")

        if (len(self.player_1) == 0) or (len(self.player_2) == 0):

            return False

        if (len(self.player_1) >= card_player_1) and (
            len(self.player_2) >= card_player_2
        ):

            return True

        return False

    def calculate_score(self, player):

        assert self.winner

        score = 0

        for idx, value in enumerate(list(reversed(player)), start=1):

            val = idx * value

            score += val

        return score

In [31]:
player_1 = Player(create_player_data(data[0])[1])

player_1.num = 1

player_2 = Player(create_player_data(data[1])[1])

player_2.num = 2

In [32]:
g = Game(player_1, player_2)

In [33]:
g.play() # 31835

Winner's score: 31835


1